# FL client Joining FL experiment 

The purpose of this notebook is to show how to start clients to participate in an FL experiment.

## Prerequisites
- The [Startup notebook](1-Startup.ipynb) has been run successfully.
- A server has been started.

### Edit Docker Script

Before starting the docker script, you need to edit it to ensure that the environments (such as dataset, GPUs, memory, ...) meet your requirement.

In [1]:
import os
from IPython.display import HTML
from multiprocessing import Process

In [2]:
client_name_1 = "org1-a"
client_name_2 = "org1-b"
workspace = "demo_workspace"

client1_startup_path = os.path.join(workspace, client_name_1, 'startup')
client2_startup_path = os.path.join(workspace, client_name_2, 'startup')

[Please click here to check the org1-a script](demo_workspace/org1-a/startup/docker.sh)

[Please click here to check the org1-b script](demo_workspace/org1-b/startup/docker.sh)

As we can see, the default data directory is `MY_DATA_DIR=/home/flclient/data/msd-data/Task09_Spleen`.

Please modify it and ensure that it equals to the actual path that contains `Task09_Spleen`. If you downloaded the dataset via using `prerpare_expr_files.sh`, `MY_DATA_DIR` can be set to `$(pwd)`. In addition, you may also need to change `GPU2USE` and add `shm-size` argument for `docker run` command.

The following is my modified script for `org1-a`:


```
#!/usr/bin/env bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"

# docker run script for FL client
# local data directory
MY_DATA_DIR=$(pwd)
# for all gpus use line below 
#GPU2USE=all 
# for 2 gpus use line below
#GPU2USE=2 
# for specific gpus as gpu#0 and gpu#2 use line below
GPU2USE='"device=0"'
# to use host network, use line below
NETARG="--net=host"
# FL clients do not need to open ports, so the following line is not needed.
#NETARG="-p 443:443 -p 8003:8003"
DOCKER_IMAGE=monai_nvflare:latest
echo "Starting docker with $DOCKER_IMAGE"
docker run --rm -it --shm-size 16G --name=org1-a --gpus=$GPU2USE -u $(id -u):$(id -g) -v /etc/passwd:/etc/passwd -v /etc/group:/etc/group -v $DIR/..:/workspace/ -v $MY_DATA_DIR:/data/:ro -w /workspace/ --ipc=host $NETARG $DOCKER_IMAGE /bin/bash
```

### Check the Connection to Server

Run telnet with the server port. A successful connection should give a message as shown below.

```
Trying 127.0.0.1...
Connected to localhost.
Escape character is '^]'.
Connection closed by foreign host.
```

In [3]:
!telnet localhost 8002

Trying 127.0.0.1...
Connected to localhost.
Escape character is '^]'.
^C
Connection closed by foreign host.


### Start Clients Docker

After modifying `docker.sh` for `org1-a` and `org1-b`, we are able to start:

In [4]:
def run_client1():
    cmd = client1_startup_path + "/docker.sh"
    print("running cmd " + cmd)
    !$cmd


def run_client2():
    cmd = client2_startup_path + "/docker.sh"
    print("running cmd " + cmd)
    !$cmd


p1 = Process(target=run_client1)
p2 = Process(target=run_client2)

p1.start()
p2.start()

running cmd demo_workspace/org1-a/startup/docker.sh
running cmd demo_workspace/org1-b/startup/docker.sh
Starting docker with monai_nvflare:latest
Starting docker with monai_nvflare:latest

== PyTorch ==

NVIDIA Release 21.04 (build 22382700)
PyTorch Version 1.9.0a0+2ecb2c7

Container image Copyright (c) 2021, NVIDIA CORPORATION.  All rights reserved.

Copyright (c) 2014-2021 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (c) 2006-2010 NEC Laboratories America (Ronan Collobert, Leon Bottou, Iain Melvin, Jason Weston)
Copyright (c) 2006      Idiap Research Institute (Samy Bengio)
Copyright (c) 2001-2004 Idiap Research Institute (Ronan Collobert, Samy Bengio, Johnny Mariethoz)
Copyright (c) 2015      Google Inc.
Copyright (c) 2015      Yangqing

### Check Started Containers

In [5]:
!docker ps -a

CONTAINER ID   IMAGE                  COMMAND                  CREATED              STATUS              PORTS     NAMES
d87439228f1a   monai_nvflare:latest   "/usr/local/bin/nvid…"   4 seconds ago        Up 3 seconds                  org1-b
fc4227a0abfd   monai_nvflare:latest   "/usr/local/bin/nvid…"   4 seconds ago        Up 3 seconds                  org1-a
a4d31af29041   monai_nvflare:latest   "/usr/local/bin/nvid…"   About a minute ago   Up About a minute             flserver


### Start Clients

To start a client, you should:

- open a terminal and enter the container named `org1-a`.
- run `start.sh` under `startup/`.

In [6]:
# You can click the following link, or manually open a new terminal.
HTML('<a href="", data-commandlinker-command="terminal:create-new"> Open a new terminal</a>')

docker exec -it org1-a bash
cd startup/
sh start.shThe commands can be:

```
docker exec -it org1-a bash
cd startup/
sh start.sh
```

A successfully started client will print logs as follow:
<br>![fl](demo_figs/enter_client_success.png)<br>

To start the second client, please open a new terminal again, and just use the same commands but for the `org1-b` Docker container:
```
docker exec -it org1-b bash
cd startup/
sh start.sh
```

After a client has been successfully started, the server side will show the following information:
<br>![fl](demo_figs/successful_regist_clients.png)<br>

### Next Steps

You have now started two client containers.
In the next notebook, [Admin Startup Notebook](4-Admin.ipynb), you'll start an admin participating in the FL experiment.